In [1]:
from pathlib import Path

import numpy as np
from in_out.array_readers_and_writers import read_3D_array
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split

import polpo.lddmm.strings as lddmm_strings
import polpo.preprocessing.pd as ppd
from polpo.model_eval import (
    R2Score,
    collect_eval_results,
)
from polpo.preprocessing.load.pregnancy import DenseMaternalCsvDataLoader
from polpo.preprocessing.mesh.io import PvReader
from polpo.preprocessing.np import FlattenButFirst
from polpo.sklearn.adapter import EvaluatedModel

No CUDA runtime is found, using CUDA_HOME='/usr'


In [2]:
OUTPUTS_DIR = Path("results") / "atlas_example"
ATLAS_DIR = OUTPUTS_DIR / "atlas"

In [3]:
cp = read_3D_array(ATLAS_DIR / lddmm_strings.cp_str)
momenta = read_3D_array(ATLAS_DIR / lddmm_strings.momenta_str)

template = PvReader()(ATLAS_DIR / lddmm_strings.template_str)

momenta = FlattenButFirst()(momenta[:-1])

momenta.shape

(26, 72)

In [4]:
data = DenseMaternalCsvDataLoader(pilot=True)()

INFO: Data has already been downloaded... using cached file ('/home/luisfpereira/.herbrain/data/maternal/raw/28Baby_Hormones.csv').


In [5]:
cleaning_pipe = ppd.DfCopy() + ppd.Drop(27, inplace=True) + ppd.ColumnToDict("gestWeek")

data_ = cleaning_pipe(data)

In [6]:
x = np.array(list(data_.values()))[:, None]
y = momenta

In [8]:
model = EvaluatedModel(
    PLSRegression(),
    R2Score(),
)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    y,
    x,
    train_size=0.8,
    shuffle=False,
)


model.fit(X_train, y_train);

In [15]:
eval_res_train = collect_eval_results(model, unnest=True, outer_key="obj_regr")

print(list(eval_res_train.keys()))

['obj_regr']


In [16]:
eval_res_train

{'obj_regr': {'r2': array([0.79036156])}}

In [17]:
model.predict_eval(X_test, y_test);

In [18]:
eval_res_test = collect_eval_results(
    model, unnest=True, outer_key="obj_regr", train=False
)

print(list(eval_res_test.keys()))

['obj_regr']


In [19]:
eval_res_test

{'obj_regr': {'r2': array([-1.94991785])}}